# Recommendation Systems: Teaching AI to Know What You Like

From Netflix to Spotify, RL powers personalized recommendations!

## What You'll Learn

By the end of this notebook, you'll understand:
- The DJ analogy: how RL learns user preferences
- Multi-armed bandits: the foundation of recommendation RL
- Exploration vs exploitation: the core tradeoff
- UCB and Thompson Sampling: smart exploration
- Contextual bandits: personalized recommendations
- Full RL for sequential recommendations

**Prerequisites:** Game playing and robotics notebooks

**Time:** ~35 minutes

---
## The Big Picture: The DJ Analogy

```
    ┌────────────────────────────────────────────────────────────────┐
    │          THE DJ ANALOGY                                        │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  Imagine you're a DJ at a party...                            │
    │                                                                │
    │  YOUR GOAL:                                                   │
    │    Keep the crowd dancing (maximize engagement)              │
    │                                                                │
    │  THE CHALLENGE:                                               │
    │    You don't know what songs they like!                      │
    │                                                                │
    │  EXPLOIT (play safe):                                        │
    │    Play songs that worked before                             │
    │    "They liked pop, keep playing pop"                        │
    │    → Might miss songs they'd love even more!                │
    │                                                                │
    │  EXPLORE (take risks):                                       │
    │    Try new genres, new artists                               │
    │    "Let's see if they like jazz"                             │
    │    → Might find hidden gems or clear the floor!             │
    │                                                                │
    │  THE RL CONNECTION:                                          │
    │    Songs = Actions (what to recommend)                       │
    │    Dancing = Reward (user engagement)                        │
    │    Balance explore/exploit to maximize total dancing!        │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Circle, Rectangle, Wedge
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

In [ ]:
# Visualize the explore-exploit dilemma

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: Pure exploitation
ax1 = axes[0]
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.axis('off')
ax1.set_title('Pure EXPLOITATION', fontsize=14, fontweight='bold', color='#1976d2')

# Known good option
box1 = FancyBboxPatch((2, 4), 2.5, 4, boxstyle="round,pad=0.1",
                       facecolor='#4caf50', edgecolor='#2e7d32', linewidth=3)
ax1.add_patch(box1)
ax1.text(3.25, 7, 'Pop Music', ha='center', fontsize=10, fontweight='bold', color='white')
ax1.text(3.25, 6, '70% like', ha='center', fontsize=9, color='white')
ax1.text(3.25, 5, '(Known)', ha='center', fontsize=8, color='white')

# Unknown options (grayed out)
for i, (x, name) in enumerate([(5.5, 'Jazz?'), (5.5, 'Rock?'), (5.5, 'Classical?')]):
    y = 6.5 - i * 2
    box = FancyBboxPatch((x, y), 2, 1.3, boxstyle="round,pad=0.1",
                          facecolor='#bdbdbd', edgecolor='#757575', linewidth=1, alpha=0.5)
    ax1.add_patch(box)
    ax1.text(x + 1, y + 0.65, name, ha='center', fontsize=8, color='#666')

ax1.text(5, 1.5, 'Safe but limited!', ha='center', fontsize=11, style='italic', color='#666')
ax1.text(5, 0.8, 'What if Jazz is 90%?', ha='center', fontsize=10, color='#d32f2f')

# Right: Pure exploration
ax2 = axes[1]
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 10)
ax2.axis('off')
ax2.set_title('Pure EXPLORATION', fontsize=14, fontweight='bold', color='#388e3c')

# Random sampling
genres = ['Pop', 'Jazz', 'Rock', 'Classical']
colors = ['#4caf50', '#2196f3', '#ff9800', '#9c27b0']
for i, (genre, color) in enumerate(zip(genres, colors)):
    x = 2 + (i % 2) * 3
    y = 5 + (i // 2) * 2.5
    box = FancyBboxPatch((x, y), 2.5, 1.8, boxstyle="round,pad=0.1",
                          facecolor=color, edgecolor='black', linewidth=2)
    ax2.add_patch(box)
    ax2.text(x + 1.25, y + 1.2, genre, ha='center', fontsize=9, fontweight='bold', color='white')
    ax2.text(x + 1.25, y + 0.5, '???', ha='center', fontsize=9, color='white')

ax2.text(5, 3, 'Random!', ha='center', fontsize=12, fontweight='bold', color='#666')
ax2.text(5, 1.5, 'Learn everything but...', ha='center', fontsize=11, style='italic', color='#666')
ax2.text(5, 0.8, 'Waste time on bad options!', ha='center', fontsize=10, color='#d32f2f')

plt.tight_layout()
plt.show()

print("\nTHE DILEMMA:")
print("  Exploit too much → Might miss better options")
print("  Explore too much → Waste time on bad options")
print("  SOLUTION: Multi-armed bandits balance both!")

---
## Multi-Armed Bandits: The Foundation

```
    ┌────────────────────────────────────────────────────────────────┐
    │          MULTI-ARMED BANDIT                                    │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  Imagine a casino with K slot machines ("one-armed bandits")  │
    │                                                                │
    │      [🎰]    [🎰]    [🎰]    [🎰]    [🎰]                      │
    │      Arm 1   Arm 2   Arm 3   Arm 4   Arm 5                    │
    │       ???     ???     ???     ???     ???                      │
    │                                                                │
    │  EACH ARM:                                                    │
    │    Has an unknown probability of paying out                  │
    │    You learn by pulling and observing                        │
    │                                                                │
    │  YOUR GOAL:                                                   │
    │    Maximize total winnings over T pulls                      │
    │                                                                │
    │  IN RECOMMENDATIONS:                                          │
    │    Arms = Products/Videos/Songs to recommend                 │
    │    Reward = User clicks, purchases, engagement               │
    │    Goal = Maximize user satisfaction                         │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
class MultiArmedBanditEnvironment:
    """
    Simulates a multi-armed bandit problem.
    
    Each arm has a fixed probability of giving reward.
    This is like different products having different click rates.
    """
    
    def __init__(self, n_arms=5, seed=42):
        np.random.seed(seed)
        self.n_arms = n_arms
        # True probabilities (unknown to the agent)
        self.true_probs = np.random.uniform(0.2, 0.8, n_arms)
        self.best_arm = np.argmax(self.true_probs)
        
    def pull(self, arm):
        """Pull an arm and get reward (0 or 1)."""
        return 1 if np.random.random() < self.true_probs[arm] else 0
    
    def get_optimal_value(self):
        """Return the value of always picking the best arm."""
        return self.true_probs[self.best_arm]


# Create environment
env = MultiArmedBanditEnvironment(n_arms=5)

print("MULTI-ARMED BANDIT ENVIRONMENT")
print("="*50)
print("\nTrue probabilities (hidden from agent):")
for i, p in enumerate(env.true_probs):
    indicator = " ← BEST" if i == env.best_arm else ""
    print(f"  Arm {i+1}: {p:.2%}{indicator}")
print(f"\nOptimal strategy would give: {env.get_optimal_value():.2%} per pull")

---
## Epsilon-Greedy: Simple but Effective

```
    ┌────────────────────────────────────────────────────────────────┐
    │          EPSILON-GREEDY ALGORITHM                              │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  THE IDEA:                                                    │
    │    With probability ε: Explore (random arm)                  │
    │    With probability 1-ε: Exploit (best arm so far)           │
    │                                                                │
    │  EXAMPLE (ε = 0.1):                                           │
    │    90% of the time: Pick the arm with highest estimated value│
    │    10% of the time: Pick a random arm                        │
    │                                                                │
    │  UPDATE RULE:                                                 │
    │    Q(a) ← Q(a) + (1/n) × (reward - Q(a))                     │
    │    Running average of rewards for each arm                   │
    │                                                                │
    │  PROS:                                                        │
    │    ✓ Simple to implement                                     │
    │    ✓ Guaranteed exploration                                  │
    │                                                                │
    │  CONS:                                                        │
    │    ✗ Explores uniformly (even clearly bad arms)              │
    │    ✗ Fixed ε doesn't adapt over time                         │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
class EpsilonGreedy:
    """
    Epsilon-Greedy bandit algorithm.
    
    The simplest exploration strategy:
    - Explore with probability epsilon
    - Exploit (pick best) with probability 1-epsilon
    """
    
    def __init__(self, n_arms, epsilon=0.1):
        self.n_arms = n_arms
        self.epsilon = epsilon
        self.reset()
        
    def reset(self):
        """Reset all estimates to zero."""
        self.counts = np.zeros(self.n_arms)  # How many times each arm pulled
        self.values = np.zeros(self.n_arms)  # Estimated value of each arm
        
    def select_arm(self):
        """Select which arm to pull."""
        if np.random.random() < self.epsilon:
            # Explore: random arm
            return np.random.randint(self.n_arms)
        else:
            # Exploit: best arm so far
            return np.argmax(self.values)
    
    def update(self, arm, reward):
        """
        Update estimate for an arm after observing reward.
        
        Uses incremental mean update:
        Q(a) = Q(a) + (1/n) * (r - Q(a))
        """
        self.counts[arm] += 1
        n = self.counts[arm]
        # Incremental update of running average
        self.values[arm] += (reward - self.values[arm]) / n
        
    def get_name(self):
        return f"ε-Greedy (ε={self.epsilon})"


# Demonstrate epsilon-greedy
print("EPSILON-GREEDY DEMONSTRATION")
print("="*50)

agent = EpsilonGreedy(n_arms=5, epsilon=0.1)
env = MultiArmedBanditEnvironment(n_arms=5)

# Run for 100 steps
total_reward = 0
for t in range(100):
    arm = agent.select_arm()
    reward = env.pull(arm)
    agent.update(arm, reward)
    total_reward += reward

print("\nAfter 100 pulls:")
print(f"  Total reward: {total_reward}")
print(f"  Average reward: {total_reward/100:.2%}")
print(f"  Optimal would be: {env.get_optimal_value():.2%}")

print("\nEstimated values vs True values:")
for i in range(5):
    print(f"  Arm {i+1}: Est={agent.values[i]:.2%}, True={env.true_probs[i]:.2%}, Pulls={int(agent.counts[i])}")

---
## UCB: Upper Confidence Bound

```
    ┌────────────────────────────────────────────────────────────────┐
    │          UCB (UPPER CONFIDENCE BOUND)                          │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  THE IDEA: "Optimism in the face of uncertainty"              │
    │                                                                │
    │  FORMULA:                                                     │
    │    UCB(a) = Q(a) + c × √(ln(t) / N(a))                       │
    │                                                                │
    │    Where:                                                     │
    │    - Q(a): Estimated value of arm a                          │
    │    - N(a): Number of times arm a was pulled                  │
    │    - t: Total number of pulls so far                         │
    │    - c: Exploration coefficient                              │
    │                                                                │
    │  WHY IT WORKS:                                                │
    │    - Arms with few pulls have HIGH uncertainty bonus         │
    │    - As arm is pulled more, bonus SHRINKS                    │
    │    - Naturally balances explore/exploit!                     │
    │                                                                │
    │  VISUALIZATION:                                               │
    │    Arm 1: ████████░░░░░  Q=0.6, bonus=0.3  →  UCB=0.9       │
    │    Arm 2: ████░░░░░░░░░  Q=0.3, bonus=0.5  →  UCB=0.8       │
    │    (Arm 2 gets picked despite lower Q because uncertain!)   │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
class UCB:
    """
    Upper Confidence Bound algorithm.
    
    Key idea: Be optimistic about uncertain arms!
    Add an uncertainty bonus that decreases as we pull more.
    """
    
    def __init__(self, n_arms, c=2.0):
        self.n_arms = n_arms
        self.c = c  # Exploration coefficient
        self.reset()
        
    def reset(self):
        self.counts = np.zeros(self.n_arms)
        self.values = np.zeros(self.n_arms)
        self.total_pulls = 0
        
    def select_arm(self):
        """Select arm with highest UCB value."""
        self.total_pulls += 1
        
        # First, pull each arm once to initialize
        for arm in range(self.n_arms):
            if self.counts[arm] == 0:
                return arm
        
        # Compute UCB for each arm
        ucb_values = np.zeros(self.n_arms)
        for arm in range(self.n_arms):
            # Uncertainty bonus: decreases as we pull more
            bonus = self.c * np.sqrt(np.log(self.total_pulls) / self.counts[arm])
            ucb_values[arm] = self.values[arm] + bonus
            
        return np.argmax(ucb_values)
    
    def update(self, arm, reward):
        """Update estimate after observing reward."""
        self.counts[arm] += 1
        n = self.counts[arm]
        self.values[arm] += (reward - self.values[arm]) / n
        
    def get_name(self):
        return f"UCB (c={self.c})"


# Demonstrate UCB
print("UCB DEMONSTRATION")
print("="*50)

agent = UCB(n_arms=5, c=2.0)
env = MultiArmedBanditEnvironment(n_arms=5)

# Run for 100 steps
total_reward = 0
for t in range(100):
    arm = agent.select_arm()
    reward = env.pull(arm)
    agent.update(arm, reward)
    total_reward += reward

print("\nAfter 100 pulls:")
print(f"  Total reward: {total_reward}")
print(f"  Average reward: {total_reward/100:.2%}")

print("\nPull distribution (UCB focuses on promising arms):")
for i in range(5):
    bar = '█' * int(agent.counts[i] / 2)
    best = " ← BEST" if i == env.best_arm else ""
    print(f"  Arm {i+1}: {bar} ({int(agent.counts[i])} pulls){best}")

---
## Thompson Sampling: The Bayesian Approach

```
    ┌────────────────────────────────────────────────────────────────┐
    │          THOMPSON SAMPLING                                     │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  THE IDEA: Maintain probability distributions, sample!        │
    │                                                                │
    │  FOR EACH ARM:                                                │
    │    Keep track of: successes (α) and failures (β)             │
    │    Belief: Probability follows Beta(α, β) distribution       │
    │                                                                │
    │  TO SELECT AN ARM:                                            │
    │    1. Sample θ̃ᵢ ~ Beta(αᵢ, βᵢ) for each arm                │
    │    2. Pick arm with highest sample: argmax θ̃ᵢ                │
    │                                                                │
    │  WHY IT WORKS:                                                │
    │    - Uncertain arms have wide distributions                  │
    │    - Sometimes sample high → explore!                        │
    │    - Confident good arms sample high often → exploit!        │
    │    - Naturally balances based on uncertainty!                │
    │                                                                │
    │  ADVANTAGE:                                                   │
    │    Often achieves BEST empirical performance!                │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
class ThompsonSampling:
    """
    Thompson Sampling algorithm.
    
    Bayesian approach: maintain belief about each arm,
    sample from beliefs to select actions.
    """
    
    def __init__(self, n_arms):
        self.n_arms = n_arms
        self.reset()
        
    def reset(self):
        # Beta distribution parameters: starts with Beta(1,1) = uniform
        self.alpha = np.ones(self.n_arms)  # Successes + 1
        self.beta = np.ones(self.n_arms)   # Failures + 1
        self.counts = np.zeros(self.n_arms)
        
    def select_arm(self):
        """Sample from each arm's distribution, pick highest."""
        # Sample from Beta distribution for each arm
        samples = np.random.beta(self.alpha, self.beta)
        return np.argmax(samples)
    
    def update(self, arm, reward):
        """Update belief after observing reward."""
        self.counts[arm] += 1
        if reward == 1:
            self.alpha[arm] += 1
        else:
            self.beta[arm] += 1
            
    def get_name(self):
        return "Thompson Sampling"
    
    @property
    def values(self):
        """Return expected value (mean of Beta)."""
        return self.alpha / (self.alpha + self.beta)


# Demonstrate Thompson Sampling
print("THOMPSON SAMPLING DEMONSTRATION")
print("="*50)

agent = ThompsonSampling(n_arms=5)
env = MultiArmedBanditEnvironment(n_arms=5)

# Run for 100 steps
total_reward = 0
for t in range(100):
    arm = agent.select_arm()
    reward = env.pull(arm)
    agent.update(arm, reward)
    total_reward += reward

print("\nAfter 100 pulls:")
print(f"  Total reward: {total_reward}")
print(f"  Average reward: {total_reward/100:.2%}")

print("\nBelief distributions (α, β):")
for i in range(5):
    mean = agent.alpha[i] / (agent.alpha[i] + agent.beta[i])
    print(f"  Arm {i+1}: α={agent.alpha[i]:.0f}, β={agent.beta[i]:.0f}, Mean={mean:.2%}, True={env.true_probs[i]:.2%}")

In [ ]:
# Visualize Thompson Sampling beliefs

from scipy import stats

fig, axes = plt.subplots(1, 5, figsize=(15, 4))
fig.suptitle('Thompson Sampling: Belief Distributions After 100 Pulls', fontsize=14, fontweight='bold')

x = np.linspace(0, 1, 100)

for i, ax in enumerate(axes):
    # Plot Beta distribution
    y = stats.beta.pdf(x, agent.alpha[i], agent.beta[i])
    color = '#4caf50' if i == env.best_arm else '#2196f3'
    ax.fill_between(x, y, alpha=0.3, color=color)
    ax.plot(x, y, color=color, linewidth=2)
    
    # Mark true probability
    ax.axvline(env.true_probs[i], color='red', linestyle='--', linewidth=2, label='True')
    
    title = f'Arm {i+1}'
    if i == env.best_arm:
        title += ' (BEST)'
    ax.set_title(title, fontsize=10)
    ax.set_xlabel('Probability', fontsize=9)
    ax.set_xlim(0, 1)
    ax.grid(True, alpha=0.3)

axes[0].set_ylabel('Density', fontsize=10)
axes[-1].legend(loc='upper right', fontsize=8)

plt.tight_layout()
plt.show()

print("\nKEY INSIGHT:")
print("  - Peaked distributions = confident (many pulls)")
print("  - Wide distributions = uncertain (few pulls)")
print("  - Red dashed line = true probability")

---
## Algorithm Comparison: Who Wins?

Let's compare all three algorithms!

In [ ]:
def run_experiment(algorithm_class, env, n_steps=1000, **kwargs):
    """
    Run a bandit experiment and track cumulative reward.
    """
    agent = algorithm_class(n_arms=env.n_arms, **kwargs)
    rewards = []
    
    for t in range(n_steps):
        arm = agent.select_arm()
        reward = env.pull(arm)
        agent.update(arm, reward)
        rewards.append(reward)
    
    return np.cumsum(rewards), agent.get_name()


def compute_regret(cumulative_rewards, optimal_value, n_steps):
    """
    Compute regret: how much worse than optimal.
    
    Regret = optimal_total - actual_total
    """
    optimal_rewards = optimal_value * np.arange(1, n_steps + 1)
    return optimal_rewards - cumulative_rewards


# Run comparison
print("ALGORITHM COMPARISON")
print("="*60)

n_steps = 1000
env = MultiArmedBanditEnvironment(n_arms=10)  # 10 products
optimal = env.get_optimal_value()

algorithms = [
    (EpsilonGreedy, {'epsilon': 0.1}),
    (EpsilonGreedy, {'epsilon': 0.01}),
    (UCB, {'c': 2.0}),
    (ThompsonSampling, {}),
]

results = []
for algo_class, kwargs in algorithms:
    cum_rewards, name = run_experiment(algo_class, env, n_steps, **kwargs)
    regret = compute_regret(cum_rewards, optimal, n_steps)
    results.append((name, cum_rewards, regret))
    print(f"  {name}: Total reward = {cum_rewards[-1]:.0f}, Final regret = {regret[-1]:.1f}")

print(f"\nOptimal total would be: {optimal * n_steps:.0f}")

In [ ]:
# Visualize comparison

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
colors = ['#f44336', '#ff9800', '#4caf50', '#2196f3']

# Left: Cumulative reward
ax1 = axes[0]
for (name, cum_rewards, regret), color in zip(results, colors):
    ax1.plot(cum_rewards, label=name, color=color, linewidth=2)

# Plot optimal
ax1.plot(optimal * np.arange(1, n_steps + 1), 'k--', label='Optimal', linewidth=2)

ax1.set_xlabel('Time Step', fontsize=11)
ax1.set_ylabel('Cumulative Reward', fontsize=11)
ax1.set_title('Cumulative Reward Over Time', fontsize=12, fontweight='bold')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Right: Regret
ax2 = axes[1]
for (name, cum_rewards, regret), color in zip(results, colors):
    ax2.plot(regret, label=name, color=color, linewidth=2)

ax2.set_xlabel('Time Step', fontsize=11)
ax2.set_ylabel('Cumulative Regret', fontsize=11)
ax2.set_title('Regret Over Time (lower is better)', fontsize=12, fontweight='bold')
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKEY INSIGHTS:")
print("  • Thompson Sampling often has lowest regret")
print("  • UCB provides good theoretical guarantees")
print("  • ε-Greedy is simple but competitive")
print("  • All algorithms improve with more data!")

---
## Contextual Bandits: Personalized Recommendations

```
    ┌────────────────────────────────────────────────────────────────┐
    │          CONTEXTUAL BANDITS                                    │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  PROBLEM WITH BASIC BANDITS:                                  │
    │    Same recommendation for everyone!                         │
    │    But different users have different preferences...         │
    │                                                                │
    │  CONTEXTUAL BANDITS:                                          │
    │    Observe context (user features) before choosing action    │
    │                                                                │
    │  EXAMPLE:                                                     │
    │    Context: [age=25, genre_pref=action, time=evening]        │
    │    Action: Which movie to recommend?                         │
    │    Reward: Did they watch it?                                │
    │                                                                │
    │  MODEL:                                                       │
    │    r(context, action) = context^T × θ_action                 │
    │    Linear model predicting reward from context               │
    │                                                                │
    │  ALGORITHMS:                                                  │
    │    • LinUCB: UCB with linear reward model                    │
    │    • Linear Thompson Sampling                                │
    │    • Neural bandits (deep learning)                          │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
class LinearContextualBandit:
    """
    Simple contextual bandit with linear reward model.
    
    For each arm, learns: reward = context @ weights
    Uses Ridge regression for stability.
    """
    
    def __init__(self, n_arms, context_dim, alpha=1.0):
        self.n_arms = n_arms
        self.context_dim = context_dim
        self.alpha = alpha  # Exploration parameter
        
        # Initialize for each arm
        self.A = [np.eye(context_dim) for _ in range(n_arms)]  # Design matrix
        self.b = [np.zeros(context_dim) for _ in range(n_arms)]  # Reward vector
        
    def select_arm(self, context):
        """Select arm using LinUCB strategy."""
        ucb_values = np.zeros(self.n_arms)
        
        for arm in range(self.n_arms):
            A_inv = np.linalg.inv(self.A[arm])
            theta = A_inv @ self.b[arm]
            
            # UCB: mean + uncertainty
            mean = context @ theta
            uncertainty = self.alpha * np.sqrt(context @ A_inv @ context)
            ucb_values[arm] = mean + uncertainty
            
        return np.argmax(ucb_values)
    
    def update(self, arm, context, reward):
        """Update model after observing reward."""
        self.A[arm] += np.outer(context, context)
        self.b[arm] += reward * context


# Simulate contextual recommendation
print("CONTEXTUAL BANDIT: MOVIE RECOMMENDATIONS")
print("="*60)

# Context features: [age_young, likes_action, evening_time]
# 4 movies to recommend
n_arms = 4
context_dim = 3
movie_names = ['Action Blockbuster', 'Romantic Comedy', 'Documentary', 'Horror']

# True preferences (hidden)
# Different user types prefer different movies!
true_weights = np.array([
    [0.8, 0.9, 0.3],   # Action: young + action fans + any time
    [0.3, 0.1, 0.7],   # Romance: older + non-action + evening
    [0.4, 0.1, 0.2],   # Documentary: moderate preferences
    [0.6, 0.5, 0.8],   # Horror: young + evening
])

agent = LinearContextualBandit(n_arms, context_dim, alpha=0.5)

# Simulate users
total_reward = 0
n_users = 500

for _ in range(n_users):
    # Random user context
    context = np.random.uniform(0, 1, context_dim)
    
    # Agent picks a movie
    arm = agent.select_arm(context)
    
    # True reward (with noise)
    true_prob = 1 / (1 + np.exp(-context @ true_weights[arm]))
    reward = 1 if np.random.random() < true_prob else 0
    
    agent.update(arm, context, reward)
    total_reward += reward

print(f"\nAfter {n_users} users:")
print(f"  Total clicks: {total_reward}")
print(f"  Click rate: {total_reward/n_users:.1%}")

# Show learned preferences
print("\nLearned movie preferences:")
for arm in range(n_arms):
    A_inv = np.linalg.inv(agent.A[arm])
    theta = A_inv @ agent.b[arm]
    print(f"  {movie_names[arm]}:")
    print(f"    Young: {theta[0]:.2f}, Action fan: {theta[1]:.2f}, Evening: {theta[2]:.2f}")

In [ ]:
# Visualize contextual recommendations

fig, ax = plt.subplots(figsize=(12, 8))
ax.set_xlim(0, 12)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('Contextual Bandits: Personalized Recommendations', fontsize=16, fontweight='bold')

# User types
users = [
    (1, 7, 'Young Action Fan', '[1, 1, 0]', 'Action\nBlockbuster', '#f44336'),
    (5.5, 7, 'Older Evening\nViewer', '[0, 0, 1]', 'Romantic\nComedy', '#e91e63'),
    (10, 7, 'Young Night Owl', '[1, 0, 1]', 'Horror', '#9c27b0'),
]

for x, y, user_type, context, rec, color in users:
    # User box
    user_box = FancyBboxPatch((x-0.8, y), 1.6, 2, boxstyle="round,pad=0.1",
                               facecolor='#e3f2fd', edgecolor='#1976d2', linewidth=2)
    ax.add_patch(user_box)
    ax.text(x, y+1.6, user_type, ha='center', fontsize=8, fontweight='bold')
    ax.text(x, y+0.5, context, ha='center', fontsize=7, family='monospace')
    
    # Arrow
    ax.annotate('', xy=(x, y-0.7), xytext=(x, y-0.1),
                arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
    
    # Recommendation
    rec_box = FancyBboxPatch((x-0.8, y-2.5), 1.6, 1.5, boxstyle="round,pad=0.1",
                              facecolor=color, edgecolor='black', linewidth=2, alpha=0.8)
    ax.add_patch(rec_box)
    ax.text(x, y-1.5, rec, ha='center', fontsize=8, fontweight='bold', color='white')

# Legend
ax.text(6, 3, 'Same algorithm → Different recommendations based on context!', 
        ha='center', fontsize=11, style='italic')

# Context explanation
ax.text(6, 1.5, 'Context = [age_young, likes_action, evening_time]',
        ha='center', fontsize=10, family='monospace', color='#666')

plt.tight_layout()
plt.show()

---
## Full RL for Sequential Recommendations

```
    ┌────────────────────────────────────────────────────────────────┐
    │          FULL RL: BEYOND BANDITS                               │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  BANDITS LIMITATION:                                          │
    │    - Each recommendation is independent                      │
    │    - Don't consider long-term effects                        │
    │                                                                │
    │  REALITY:                                                     │
    │    - Recommendations affect future user state                │
    │    - Showing action movies → user expects more action        │
    │    - Need to optimize for long-term engagement               │
    │                                                                │
    │  FULL RL FORMULATION:                                         │
    │    State: User history, preferences, context                 │
    │    Action: Which item to recommend                           │
    │    Reward: Immediate engagement (clicks, watch time)         │
    │    Transition: How user state changes after interaction      │
    │    Goal: Maximize long-term engagement                       │
    │                                                                │
    │  ALGORITHMS:                                                  │
    │    • DQN for recommendations                                 │
    │    • Actor-Critic methods                                    │
    │    • Offline RL (learning from logs)                         │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Simple sequential recommendation simulation

class UserState:
    """
    Simulates a user whose preferences evolve.
    
    - Users have genre preferences
    - Preferences shift based on what they watch
    - Boredom increases if same genre repeated
    """
    
    def __init__(self, n_genres=4):
        self.n_genres = n_genres
        self.reset()
        
    def reset(self):
        # Initial genre preferences (softmax normalized)
        self.preferences = np.random.dirichlet(np.ones(self.n_genres))
        self.boredom = np.zeros(self.n_genres)  # Boredom per genre
        self.history = []  # Recent recommendations
        return self.get_state()
        
    def get_state(self):
        """Return state vector."""
        return np.concatenate([self.preferences, self.boredom])
    
    def step(self, action):
        """
        User receives recommendation and responds.
        
        Returns: reward, done
        """
        # Probability of engagement
        base_prob = self.preferences[action]
        boredom_penalty = 0.3 * self.boredom[action]
        prob = max(0.1, base_prob - boredom_penalty)
        
        # User engages or not
        engaged = np.random.random() < prob
        reward = 1.0 if engaged else 0.0
        
        # Update state
        self.history.append(action)
        
        # Boredom increases for watched genre, decreases for others
        if engaged:
            self.boredom[action] = min(1, self.boredom[action] + 0.2)
            # Preferences shift slightly toward watched genre
            self.preferences[action] = min(0.5, self.preferences[action] + 0.05)
        
        # Other genres' boredom decays
        for g in range(self.n_genres):
            if g != action:
                self.boredom[g] = max(0, self.boredom[g] - 0.1)
        
        # Normalize preferences
        self.preferences = self.preferences / self.preferences.sum()
        
        # Session ends after some interactions
        done = len(self.history) >= 20
        
        return self.get_state(), reward, done


# Compare greedy vs diverse strategies
print("SEQUENTIAL RECOMMENDATION: GREEDY vs DIVERSE")
print("="*60)

def greedy_policy(state, n_genres=4):
    """Always pick highest preference."""
    return np.argmax(state[:n_genres])

def diverse_policy(state, n_genres=4):
    """Consider preferences AND boredom."""
    preferences = state[:n_genres]
    boredom = state[n_genres:]
    scores = preferences - 0.5 * boredom
    return np.argmax(scores)

# Run comparison
n_episodes = 100
n_genres = 4
genre_names = ['Action', 'Comedy', 'Drama', 'Sci-Fi']

results = {'Greedy': [], 'Diverse': []}

for policy_name, policy in [('Greedy', greedy_policy), ('Diverse', diverse_policy)]:
    for _ in range(n_episodes):
        user = UserState(n_genres)
        state = user.reset()
        total_reward = 0
        done = False
        
        while not done:
            action = policy(state, n_genres)
            state, reward, done = user.step(action)
            total_reward += reward
            
        results[policy_name].append(total_reward)

print("\nResults (average over 100 sessions):")
for name, rewards in results.items():
    print(f"  {name}: {np.mean(rewards):.2f} ± {np.std(rewards):.2f} engagements")

print("\nINSIGHT:")
print("  Diverse policy considers long-term effects (boredom)")
print("  Sometimes recommending second-best prevents boredom!")

In [ ]:
# Visualize the difference

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Reward distribution
ax1 = axes[0]
positions = [0, 1]
bp = ax1.boxplot([results['Greedy'], results['Diverse']], positions=positions,
                  patch_artist=True)
bp['boxes'][0].set_facecolor('#ff9800')
bp['boxes'][1].set_facecolor('#4caf50')

ax1.set_xticks(positions)
ax1.set_xticklabels(['Greedy', 'Diverse'])
ax1.set_ylabel('Total Engagements per Session', fontsize=11)
ax1.set_title('Engagement Comparison', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Right: Recommendation diversity
ax2 = axes[1]

# Track one session for each
diversity_data = {}
for policy_name, policy in [('Greedy', greedy_policy), ('Diverse', diverse_policy)]:
    user = UserState(n_genres)
    state = user.reset()
    actions = []
    done = False
    
    while not done:
        action = policy(state, n_genres)
        actions.append(action)
        state, reward, done = user.step(action)
    
    diversity_data[policy_name] = actions

# Plot as bar charts
x = np.arange(n_genres)
width = 0.35

greedy_counts = [diversity_data['Greedy'].count(i) for i in range(n_genres)]
diverse_counts = [diversity_data['Diverse'].count(i) for i in range(n_genres)]

ax2.bar(x - width/2, greedy_counts, width, label='Greedy', color='#ff9800')
ax2.bar(x + width/2, diverse_counts, width, label='Diverse', color='#4caf50')

ax2.set_xlabel('Genre', fontsize=11)
ax2.set_ylabel('Times Recommended', fontsize=11)
ax2.set_title('Recommendation Diversity (One Session)', fontsize=12, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(genre_names)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nKEY OBSERVATION:")
print("  Greedy policy recommends same genre repeatedly (boredom!)")
print("  Diverse policy spreads recommendations across genres")

---
## Real-World Applications

```
    ┌────────────────────────────────────────────────────────────────┐
    │              RL IN PRODUCTION RECOMMENDATION SYSTEMS           │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  NETFLIX:                                                     │
    │    • Bandit algorithms for thumbnail selection               │
    │    • RL for personalized row ordering                        │
    │    • Contextual bandits for "Because you watched"            │
    │                                                                │
    │  SPOTIFY:                                                     │
    │    • Bandits for Discover Weekly                             │
    │    • Thompson Sampling for new song recommendations          │
    │    • RL for playlist generation                              │
    │                                                                │
    │  YOUTUBE:                                                     │
    │    • Deep RL for video recommendations                       │
    │    • Balancing watch time vs diversity                       │
    │    • Long-term user satisfaction optimization                │
    │                                                                │
    │  E-COMMERCE (Amazon, etc.):                                   │
    │    • Bandits for product ranking                             │
    │    • RL for personalized pricing                             │
    │    • Sequential recommendations in cart                      │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize real-world applications

fig, ax = plt.subplots(figsize=(14, 8))
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('RL in Real-World Recommendation Systems', fontsize=16, fontweight='bold')

# Companies and their RL uses
companies = [
    (2, 7.5, 'Netflix', 'Thumbnail selection\nRow ordering', '#e50914'),
    (6, 7.5, 'Spotify', 'Discover Weekly\nNew songs', '#1db954'),
    (10, 7.5, 'YouTube', 'Video ranking\nWatch time opt.', '#ff0000'),
    (4, 4, 'Amazon', 'Product ranking\nPricing', '#ff9900'),
    (8, 4, 'TikTok', 'Feed ranking\nCreator exposure', '#000000'),
]

for x, y, name, use, color in companies:
    # Company box
    box = FancyBboxPatch((x-1.3, y-0.5), 2.6, 2.2, boxstyle="round,pad=0.1",
                          facecolor=color, edgecolor='black', linewidth=2, alpha=0.8)
    ax.add_patch(box)
    ax.text(x, y+1.2, name, ha='center', fontsize=11, fontweight='bold', color='white')
    ax.text(x, y+0.2, use, ha='center', fontsize=8, color='white')

# Common techniques
ax.text(6, 1.5, 'Common Techniques:', ha='center', fontsize=12, fontweight='bold')
techniques = ['Multi-Armed Bandits', 'Contextual Bandits', 'Deep RL', 'Offline RL']
for i, tech in enumerate(techniques):
    ax.text(2 + i * 3.2, 0.8, f'• {tech}', fontsize=10)

plt.tight_layout()
plt.show()

---
## Summary: RL for Recommendations

### Algorithm Comparison

| Algorithm | Explore Strategy | Pros | Cons |
|-----------|-----------------|------|------|
| **ε-Greedy** | Random with prob ε | Simple | Uniform exploration |
| **UCB** | Optimism bonus | Theoretical guarantees | Can over-explore |
| **Thompson** | Sample from beliefs | Often best empirical | Computationally heavier |

### When to Use What

| Scenario | Best Approach |
|----------|---------------|
| Simple A/B testing | ε-Greedy or Thompson |
| Need guarantees | UCB |
| Personalization needed | Contextual Bandits |
| Long-term optimization | Full RL (DQN, Actor-Critic) |

### Quick Reference

```python
# Simple bandit
class EpsilonGreedy:
    def select_arm(self):
        if np.random.random() < self.epsilon:
            return np.random.randint(self.n_arms)  # Explore
        return np.argmax(self.values)  # Exploit
    
    def update(self, arm, reward):
        self.counts[arm] += 1
        self.values[arm] += (reward - self.values[arm]) / self.counts[arm]
```

---
## Test Your Understanding

**1. What is the explore-exploit tradeoff?**
<details>
<summary>Click to reveal answer</summary>
The explore-exploit tradeoff is the fundamental tension in bandits:
- Exploit: Use what you know works (pick best arm so far)
- Explore: Try uncertain options to learn more

Too much exploitation → might miss better options
Too much exploration → waste time on bad options
</details>

**2. How does UCB balance exploration and exploitation?**
<details>
<summary>Click to reveal answer</summary>
UCB adds an "optimism bonus" to each arm:
UCB(a) = estimated_value(a) + uncertainty_bonus(a)

Arms with few pulls have HIGH uncertainty bonus → get explored
As arms are pulled more, bonus decreases → exploitation wins
This naturally balances the tradeoff!
</details>

**3. Why is Thompson Sampling often the best in practice?**
<details>
<summary>Click to reveal answer</summary>
Thompson Sampling maintains probability distributions (beliefs) for each arm:
- Sample from each arm's distribution
- Pick the arm with highest sample

This naturally handles uncertainty:
- Uncertain arms have wide distributions → sometimes sample high
- Confident good arms sample high often → exploitation
- Elegant balance without tuning exploration parameters!
</details>

**4. What's the difference between bandits and full RL for recommendations?**
<details>
<summary>Click to reveal answer</summary>
Bandits: Each recommendation is independent, optimize immediate reward

Full RL: Recommendations affect future user state:
- User preferences evolve
- Boredom accumulates if same type repeated
- Need to optimize long-term engagement

Full RL considers how today's recommendation affects tomorrow's user!
</details>

**5. How do contextual bandits enable personalization?**
<details>
<summary>Click to reveal answer</summary>
Contextual bandits observe user features (context) before choosing:
- Context: age, preferences, time, device, etc.
- Learn: which items work for which contexts
- Result: Different users get different recommendations!

Example: Young action fan in evening → recommend action movie
Same time, older documentary fan → recommend documentary
</details>

---
## What's Next?

Recommendation systems show RL for personalization. In the next notebook, we'll explore a detailed case study: **LLM Alignment** using RLHF!

**Continue to:** [Notebook 4: LLM Alignment Case Study](04_llm_alignment_case_study.ipynb)

---

*"The best recommendation is the one that feels like you read the user's mind!"*